# 动机

- 考虑**社交内聚性较高、彼此之间轨迹又较为相似**的个体组成的社区
- k-core的优势（计算便捷）
- k-core的不足（内聚性不足，朋友关系联系不频繁，需要通过是否一起活动过进行验证，不能代表生活中真实的亲密关系）

```latex
Rather than being sets of high cohesion, Seidman\cite{seidman1983network} characterizes $k$-cores as "seedbeds, within which cohesive subsets can precipitate out."

@article{cohen2008trusses,
  title={Trusses: Cohesive subgraphs for social network analysis},
  author={Cohen, Jonathan},
  journal={National security agency technical report},
  volume={16},
  number={3.1},
  year={2008},
  publisher={Citeseer}
}

@article{seidman1983network,
  title={Network structure and minimum degree},
  author={Seidman, Stephen B},
  journal={Social networks},
  volume={5},
  number={3},
  pages={269--287},
  year={1983},
  publisher={Elsevier}
}
```

```latex
Social networks can be rife with casual acquaintances and zombie friends who are only nominally "friends" on the online platform, but are not actually close with one another in the real world. However, through the lens of trajectory similarity, it is possible to filter intimate relationships who do frequently hang out together offline.
```


# 解决思路

1. 利用轨迹相似度筛选边
2. 利用k-core算法进行社区发现

# 相关定义

Notation|Definition
--------|----------
$G(V, E)$|a social network, an undirected graph with vertex set $V$ and edge set $E$
$d(u, v)$|the distance between two users, $u$ and $v$, within the social network $G(V, E)$ ($u, v \in V$)
$p(φ,λ,t)$|a point on a trajectory, consisting of latitude $φ$, longitude $λ$, and time $t$
$T$|a trajectory, with $T[i]$ being the $i$th point within the trajectory
$\mathbf{T}$|a trajectory dataset, with $\mathbf{T}[u]$ being the trajectory of user $u$
$s(T_1, T_2)$|the similarity between two trajectories, $T_1$ and $T_2$, under a trajectory similarity measure $s$
$NN(T, k)$|the $k$ nearest neighbors of trajectory $T$

- $u$：社交网络中的一个用户
- $d(G, u, v)$：社交网络$G$内，用户$u$、$v$之间的距离。
- $T$：轨迹数据集，其中$T[u]$为用户u的轨迹
- $τ$：轨迹
- $s(τ_1, τ_2)$：轨迹$τ_1$、$τ_2$之间的相似度，其中$s$为一轨迹相似度度量。
- $NN(\mathbf{T}, s, τ, k)$：轨迹数据集$\mathbf{T}$内，在轨迹相似度度量$s$下，轨迹$τ$的$k$-近邻组成的集合。
- $d_{NN}(\mathbf{T}, s, τ_1, τ_2)$：轨迹数据集$\mathbf{T}$内，在轨迹相似度度量$s$下，轨迹$τ_1$、$τ_2$之间的近邻距离。
  - $d_{NN}(\mathbf{T}, s, τ_1, τ_2) = \underset {k \in \mathbb{Z}}{\operatorname{argmin}} k, τ_2 \in NN(\mathbf{T}, s, τ_1, k), τ_1 \in NN(\mathbf{T}, s, τ_2, k)$

```latex
There are two criteria we can use to identify similar trajectories, $k$-nearest neighbors (K-NN), under which each trajectory is considered to be similar to the $k$ other trajectories with the highest similarity value, as well as $ε$-nearest neighbors, under which pairs of trajectories having similarity values above a threshold $ε$ are considered to be similar \cite{liu2020stccd}. However, $ε$-nearest neighbors is very sensitive to the parameter $ε$ \cite{berton2015graph, berton2018comparison}, and when used to filter edges, it may result in networks with many disconnected parts under an improper value of $ε$. Thus, $k$-nearest neighbors is a better choice. However, $k$-nearest neighbors also has its own problems, such as being a one-way relationship. Consider the situation in which a user is not a sociable person, and his or her trajectory is not very similar to any other user's trajectory. In this situation, $k$-nearest neighbors would still consider the trajectories of $k$ other users to be similar, but should these $k$ users be socially active, it is likely that for each of these users, the $k$-nearest neighbors of his or her trajectory does not include the aforementioned introverted user's trajectory. To overcome such a problem, we propose the following mutual dissimilarity measure for two trajectories based on $k$-nearest neighbors, and use such a dissimilarity measure to filter edges.

@article{liu2020stccd,
  title={STCCD: Semantic trajectory clustering based on community detection in networks},
  author={Liu, Caihong and Guo, Chonghui},
  journal={Expert Systems with Applications},
  volume={162},
  pages={113689},
  year={2020},
  publisher={Elsevier}
}

@inproceedings{berton2015graph,
  title={Graph construction for semi-supervised learning},
  author={Berton, Lilian and de Andrade Lopes, Alneu},
  booktitle={Twenty-Fourth International Joint Conference on Artificial Intelligence},
  year={2015}
}

@inproceedings{berton2018comparison,
  title={A comparison of graph construction methods for semi-supervised learning},
  author={Berton, Lilian and de Andrade Lopes, Alneu and Vega-Oliveros, Didier A},
  booktitle={2018 International Joint Conference on Neural Networks (IJCNN)},
  pages={1--8},
  year={2018},
  organization={IEEE}
}
```

Mutual Nearest Neighbor Dissimilarity. Given two trajectories, $T_1$ and $T_2$, the Mutual Nearest Neighbor Dissimilarity between $T_1 and T_2$, $d_{MNN}(T_1, T_2)$, is the minimum value of $k \in \mathbb{Z}$ such that $T_2$ is also a $k$-nearest neighbor of $T_1$, and that $T_1$ is a $k$-nearest neighbor of $T_2$. Formally, $d_{MNN}(T_1, T_2) = \underset {k \in \mathbb{Z}}{\operatorname{argmin}} k, T_2 \in NN(T_1, k), T_1 \in NN(T_2, k)$.

Problem Statement. Given (1) a social network $G(V, E)$, (2) a trajectory dataset $\mathbf{T}$ such that $\forall u \in V, \exists! T[u] \in \mathbf{T}$, (3)

## 如何筛选边？

一条边$(u, v)$存在，当且仅当$d_{NN}(\mathbf{T}, s, \mathbf{T}[u], \mathbf{T}[v]) \le m$。

我们可以通过一种基于beam search的算法筛选这样的边。

时间复杂度：

- 每个节点：$deg(v) \times O(\log m)$
- 总复杂度：$O(V + E \log m)$

空间复杂度：$O(E)$

**删边的顺序？**

# 参数

- $d_{NN}(\mathbf{T}, s, \mathbf{T}[u], \mathbf{T}[v]) \le m$中的$m$
- 利用k-core进行社区发现的参数k

# 实验设计

- 筛选边的运行时间$t$（不同的benchmark、不同的$m$、不同的$k$）
- 对比k-core，对于每个社区$G'(V', E')$：
  - 社区的内聚性（社区内任意两个个体之间在社交网络上的平均距离，$\frac{2}{|V'|(|V'| - 1)} \sum_{u, v \in V'}{d(G', u, v)}$，越小越好）
  - 社区内个体轨迹彼此之间的相似度（社区内任意两个个体轨迹之间的平均近邻距离，$\frac{1}{|V'|(|V'| - 1)} \sum_{u, v \in V'}{(d_{NN}(T, s, \mathbf{T}[u], \mathbf{T}[v]) + d_{NN}(T, s, \mathbf{T}[v], \mathbf{T}[u]))}$，越小越好）
  - 社区相对于原社区的大小
  - 社区的密度


In [1]:
from statistics import mean

In [2]:
import more_itertools
import networkx as nx
import numpy as np

In [3]:
from nested_default_dict import NestedDefaultDict
from visualization import *

In [4]:
original_graph = nx.read_adjlist('social_network/brightkite.txt')

In [5]:
graphs = {}

for m in (5, 10, 15, 20):
    graphs[m] = nx.read_adjlist(f'filtered_edges/{m}')

In [6]:
cores = NestedDefaultDict()

In [7]:
original_cores = NestedDefaultDict()

In [8]:
def connected_subgraphs(graph):
    for connected_component in nx.connected_components(graph):
        yield graph.subgraph(connected_component)

In [9]:
def ratios_of_strangers(graph):
    number_of_nodes = len(graph)
    for node in graph.nodes:
        number_of_neighbors = len(graph[node])
        yield (number_of_nodes - number_of_neighbors - 1) / (number_of_nodes)

In [10]:
def all_pair_nearest_neighbor_distances_from_file(query_nodes, filepath):
    with open(filepath, 'r') as fp:
        for line in fp:
            if line:
                node, *nearest_neighbors = line.split()
                if node in query_nodes:
                    for (distance_minus_one, nearest_neighbor) in enumerate(nearest_neighbors):
                        if nearest_neighbor in query_nodes:
                            # yield (node, nearest_neighbor), distance_minus_one + 1
                            yield distance_minus_one + 1

In [11]:
def all_pairs_shortest_path_length(graph):
    for node, all_pairs_shortest_path_length_from_node in nx.all_pairs_shortest_path_length(graph):
        yield from all_pairs_shortest_path_length_from_node.values()

## k = 3

In [12]:
original_cores[3] = nx.k_core(original_graph, 3)

In [13]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(original_cores[3])
        )
    )
)

3.0194061335562776

In [13]:
nx.density(original_cores[3])

0.01694587295228284

In [14]:
nx.average_clustering(original_cores[3])

0.41398591223824444

In [14]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(original_cores[3])
        )
    )
)

811.5985999783887

In [16]:
cores[5][3] = nx.k_core(graphs[5], 3)
visualize_social_network_and_emphasize_communities(cores[5][3]).render_notebook()

In [16]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(cores[5][3])
        )
    )
)

1.8054711246200608

In [17]:
nx.density(cores[5][3])

0.09176788124156546

In [18]:
nx.average_clustering(cores[5][3])

0.7427350427350428

In [17]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(cores[5][3])
        )
    )
)

21.66551724137931

## k = 4

In [19]:
original_cores[4] = nx.k_core(original_graph, 4)

In [19]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(original_cores[4])
        )
    )
)

2.925668248279854

In [20]:
nx.density(original_cores[4])

0.02065474355021071

In [21]:
nx.average_clustering(original_cores[4])

0.4165286339936578

In [20]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(original_cores[4])
        )
    )
)

804.9643526516936

In [22]:
cores[5][4] = nx.k_core(graphs[5], 4)
visualize_social_network_and_emphasize_communities(cores[5][4]).render_notebook()

In [23]:
cores[10][4] = nx.k_core(graphs[10], 4)
visualize_social_network_and_emphasize_communities(cores[10][4]).render_notebook()

In [24]:
nx.density(cores[10][4])

0.047870967741935486

In [25]:
nx.average_clustering(cores[10][4])

0.47589206349206375

In [23]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(cores[10][4])
        )
    )
)

3.804513618677043

In [24]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(cores[10][4])
        )
    )
)

313.7185714285714

## k = 5

In [26]:
original_cores[5] = nx.k_core(original_graph, 5)

In [26]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(original_cores[5])
        )
    )
)

2.8431362010780354

In [27]:
nx.density(original_cores[5])

0.025447609996898244

In [28]:
nx.average_clustering(original_cores[5])

0.4248838046664066

In [27]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(original_cores[5])
        )
    )
)

790.4356914516809

In [29]:
cores[10][5] = nx.k_core(graphs[10], 5)
visualize_social_network_and_emphasize_communities(cores[10][5]).render_notebook()

In [29]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(cores[10][5])
        )
    )
)

1.25

In [30]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(cores[10][5])
        )
    )
)

62.72727272727273

## k = 6

In [31]:
original_cores[6] = nx.k_core(original_graph, 6)

In [32]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(original_cores[6])
        )
    )
)

2.7887385190037492

In [33]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(original_cores[6])
        )
    )
)

776.7600532489388

In [34]:
cores[10][6] = nx.k_core(graphs[10], 6)
visualize_social_network_and_emphasize_communities(cores[10][6]).render_notebook()

In [35]:
cores[15][6] = nx.k_core(graphs[15], 6)
visualize_social_network_and_emphasize_communities(cores[15][6]).render_notebook()

In [36]:
mean(
    more_itertools.collapse(
        (
            all_pairs_shortest_path_length(connected_subgraph)
            for connected_subgraph in connected_subgraphs(cores[15][6])
        )
    )
)

2.3369597177300796

In [37]:
mean(
    more_itertools.collapse(
        (
            all_pair_nearest_neighbor_distances_from_file(connected_subgraph.nodes, 'all_pair_nearest_neighbor_distances')
            for connected_subgraph in connected_subgraphs(cores[15][6])
        )
    )
)

249.91837968561063